# 5. szekció – Többügynökös Orkesztrátor

Egy egyszerű kétügynökös folyamatot mutat be (Kutató -> Szerkesztő) a Foundry Local használatával.


### Magyarázat: Függőségek telepítése
Telepíti a `foundry-local-sdk` és `openai` csomagokat, amelyek szükségesek a helyi modell eléréséhez és a csevegési műveletekhez. Idempotens.


# Forgatókönyv
Egy minimális két-ügynökös orkestrációs mintát valósít meg:
- **Kutató ügynök** tömör, tényszerű pontokat gyűjt
- **Szerkesztő ügynök** átírja ezeket vezetői érthetőségre

Bemutatja az ügynökönként megosztott memóriát, a köztes kimenet szekvenciális továbbítását és egy egyszerű csővezeték-funkciót. Kiterjeszthető további szerepekre (pl. Kritikus, Ellenőr) vagy párhuzamos ágakra.

**Környezeti változók:**
- `FOUNDRY_LOCAL_ALIAS` - Alapértelmezett modell használata (alapértelmezett: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Elsődleges ügynök modell (felülírja az ALIAS-t)
- `AGENT_MODEL_EDITOR` - Szerkesztő ügynök modell (alapértelmezés szerint az elsődleges)

**SDK Referencia:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Hogyan működik:**
1. **FoundryLocalManager** automatikusan elindítja a Foundry Local szolgáltatást
2. Letölti és betölti a megadott modellt (vagy a gyorsítótárban lévőt használja)
3. OpenAI-kompatibilis végpontot biztosít az interakcióhoz
4. Minden ügynök különböző modellt használhat a specializált feladatokhoz
5. Beépített újrapróbálkozási logika kezeli az átmeneti hibákat

**Főbb jellemzők:**
- ✅ Automatikus szolgáltatás-felfedezés és inicializálás
- ✅ Modell életciklus-kezelés (letöltés, gyorsítótárazás, betöltés)
- ✅ OpenAI SDK kompatibilitás a megszokott API-hoz
- ✅ Többmodell-támogatás az ügynökök specializációjához
- ✅ Robusztus hibakezelés újrapróbálkozási logikával
- ✅ Helyi következtetés (nincs szükség felhő API-ra)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Magyarázat: Alapvető importok és típusmegadás
Bevezeti a dataclassokat az ügynök üzeneteinek tárolására, valamint a típusmegadásokat az érthetőség érdekében. Importálja a Foundry Local managert és az OpenAI klienst a későbbi ügynöki műveletekhez.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Magyarázat: Modell inicializálása (SDK minta)
A Foundry Local Python SDK-t használja a robusztus modellkezeléshez:
- **FoundryLocalManager(alias)** - Automatikusan elindítja a szolgáltatást és betölti a modellt az alias alapján
- **get_model_info(alias)** - Az alias-t konkrét modellazonosítóra oldja fel
- **manager.endpoint** - Biztosítja a szolgáltatás végpontját az OpenAI kliens számára
- **manager.api_key** - API kulcsot biztosít (helyi használathoz opcionális)
- Támogatja külön modellek használatát különböző ügynökök számára (elsődleges vs szerkesztő)
- Beépített újrapróbálkozási logika exponenciális visszaeséssel a rugalmasság érdekében
- Kapcsolatellenőrzés a szolgáltatás készenlétének biztosítására

**Kulcsfontosságú SDK minta:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Életciklus-kezelés:**
- A menedzserek globálisan tárolódnak a megfelelő tisztítás érdekében
- Minden ügynök külön modellt használhat a specializációhoz
- Automatikus szolgáltatásfelismerés és kapcsolatkezelés
- Kíméletes újrapróbálkozás exponenciális visszaeséssel hiba esetén

Ez biztosítja a megfelelő inicializálást, mielőtt az ügynökök összehangolása megkezdődik.

**Hivatkozás:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Magyarázat: Agent és Memory osztályok
Egy könnyű `AgentMsg` osztályt definiál a memória bejegyzésekhez, valamint egy `Agent` osztályt, amely magában foglalja:
- **Rendszer szerep** - Az ügynök személyisége és utasításai
- **Üzenet történet** - A beszélgetés kontextusának fenntartása
- **act() metódus** - Műveletek végrehajtása megfelelő hibakezeléssel

Az ügynök különböző modelleket használhat (elsődleges vs szerkesztő), és minden ügynök számára elkülönített kontextust tart fenn. Ez a minta lehetővé teszi:
- Memória megőrzését a műveletek között
- Rugalmas modell hozzárendelést ügynökönként
- Hibák elkülönítését és helyreállítását
- Egyszerű láncolást és összehangolást


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Magyarázat: Orkesztrált Adatfeldolgozó Folyamat
Két specializált ügynököt hoz létre:
- **Kutató**: Az elsődleges modellt használja, tényeken alapuló információkat gyűjt
- **Szerkesztő**: Külön modell használatára is képes (ha be van állítva), finomítja és átfogalmazza az adatokat

A `pipeline` függvény:
1. A Kutató nyers információkat gyűjt
2. A Szerkesztő végleges, vezetői szintű kimenetté alakítja
3. Visszaadja mind a köztes, mind a végleges eredményeket

Ez a minta lehetővé teszi:
- Modell specializációt (különböző modellek különböző szerepekhez)
- Minőségjavítást többlépcsős feldolgozással
- Az információ átalakulásának nyomon követhetőségét
- Egyszerű bővítést további ügynökökkel vagy párhuzamos feldolgozással


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Magyarázat: Folyamat végrehajtása és eredmények
A többügynökös folyamatot egy megfelelőségi + késleltetési témájú kérdésen hajtja végre, hogy bemutassa:
- Több szakaszos információátalakítás
- Ügynökök specializációja és együttműködése
- Az output minőségének javítása finomítással
- Nyomon követhetőség (mind a köztes, mind a végső eredmények megőrzése)

**Eredménystruktúra:**
- `question` - Eredeti felhasználói kérdés
- `research` - Nyers kutatási eredmény (tények felsorolása)
- `final` - Finomított vezetői összefoglaló
- `models` - Mely modellek kerültek felhasználásra az egyes szakaszokban

**Bővítési ötletek:**
1. Kritikus ügynök hozzáadása a minőségellenőrzéshez
2. Párhuzamos kutatóügynökök alkalmazása különböző aspektusokhoz
3. Hitelesítő ügynök hozzáadása a tényellenőrzéshez
4. Különböző modellek használata eltérő komplexitási szintekhez
5. Visszacsatolási hurkok bevezetése az iteratív fejlesztéshez


### Haladó: Egyedi ügynök konfiguráció

Próbálja meg testre szabni az ügynök viselkedését a környezeti változók módosításával, mielőtt futtatná az inicializáló cellát:

**Elérhető modellek:**
- Használja a `foundry model ls` parancsot a terminálban az összes elérhető modell megtekintéséhez
- Példák: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b stb.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Felelősség kizárása**:  
Ezt a dokumentumot az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével fordították le. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt a professzionális emberi fordítás igénybevétele. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
